In [3]:
%matplotlib inline

In [4]:
# Author: Yulou Zhou
#Weight loss function by commonality (and normalize)
#padded sequence
#for loop to test parameters
#for loop to get prev generated chord


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import csv
import glob
import os
import pandas as pd
import numpy as np
import csv
from random import shuffle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from MyDataset import MyDataset
torch.manual_seed(1)

In [5]:
batch_size = 4
train_data = MyDataset('relativedata', prevchord = 4)
weights = train_data.stats()
data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
c=0
for i in data_loader:
    print("p: ", i['p'])
    print("m: ", i['m'])
    print("l: ", i['l'])
    print("mask: ", i['mask'])
    c += 10
    if c > 0: break

p:  tensor([[  5,   4,   7,  10,   3,   7,  10,   3,   7,  10,   3,   7],
        [  6,   4,   7,   6,   4,   7,  11,   4,   7,   2,   4,   7],
        [  6,   4,   7,   8,   3,   7,   6,   4,   7,   6,   4,   7],
        [  6,   2,   7,   1,   7,   0,   8,   4,   7,  10,   3,   7]])
m:  tensor([[ 10,  10,  10,   0,   1,   5,  10,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 12,  12,  12,  12,  12,  11,  12,  12,  12,  12,  12,  12,
          12,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 10,  10, 

In [6]:
def normalize(array):
    return [i / sum(array) for i in array]
newweights = []

def weightloss(weights, eps):
    for w in weights:
        newweights.append(torch.FloatTensor( normalize([1/(i + eps) for i in w] ) ) )
    print(newweights)


In [29]:
import random

P_HIDDEN_DIM = max(int(np.random.normal(30, 30)), 5)
M_HIDDEN_DIM = max(int(np.random.normal(30, 30)), 5)
vocab_size = 13
tagset_size = 13
EPS = random.uniform(0.005, 0.15)
ITER = 10000
P_EMBEDDING_DIM = vocab_size
M_EMBEDDING_DIM = vocab_size

print("iter: ", ITER, "\np_hidden: ", P_HIDDEN_DIM, "\nm_hidden: ", M_HIDDEN_DIM, "\neps: ", EPS)

iter:  10000 
p_hidden:  16 
m_hidden:  43 
eps:  0.14212619713184405


In [30]:
newweishts = weightloss(weights, EPS)

[tensor([ 0.1083,  0.0468,  0.0985,  0.0589,  0.0788,  0.0805,  0.0526,
         0.1018,  0.0502,  0.0907,  0.0684,  0.0690,  0.0956]), tensor(1.00000e-02 *
       [ 8.4815,  8.7604,  8.6949,  3.1158,  1.3336,  8.3771,  8.7604,
         8.6741,  8.7604,  8.7604,  8.7604,  8.7604,  8.7604]), tensor(1.00000e-02 *
       [ 8.1590,  8.5086,  8.5086,  8.4634,  8.5086,  8.5077,  7.3920,
         1.0563,  8.1425,  7.2278,  8.5086,  8.5086,  8.5086]), tensor([ 0.1068,  0.0477,  0.0976,  0.0596,  0.0789,  0.0806,  0.0535,
         0.1007,  0.0510,  0.0903,  0.0689,  0.0695,  0.0949]), tensor(1.00000e-02 *
       [ 8.4771,  8.7395,  8.6779,  3.2311,  1.4004,  8.3787,  8.7395,
         8.6584,  8.7395,  8.7395,  8.7395,  8.7395,  8.7395]), tensor(1.00000e-02 *
       [ 8.1606,  8.4896,  8.4896,  8.4472,  8.4896,  8.4888,  7.4330,
         1.1113,  8.1451,  7.2763,  8.4896,  8.4896,  8.4896])]


Create the model:



In [34]:
class LSTMTagger(nn.Module):

    def __init__(self, p_embed_dim, m_embed_dim, p_hidden_dim, m_hidden_dim, vocab_size, tagset_size, batch_size = 1):
        super(LSTMTagger, self).__init__()
        self.batch_size = batch_size
        
        self.p_embed_dim, self.m_embed_dim = p_embed_dim, m_embed_dim
        #refer to https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222/2
        
        self.p_embed = nn.Embedding(vocab_size, p_embed_dim)
        self.m_embed = nn.Embedding(vocab_size, m_embed_dim)
        self.p_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        self.m_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.p_lstm = nn.LSTM(p_embed_dim, p_hidden_dim, bidirectional = False)
        self.m_lstm = nn.LSTM(m_embed_dim, m_hidden_dim, bidirectional = False)
        

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear((p_hidden_dim + m_hidden_dim) , tagset_size)
        self.root2second = nn.Linear((p_hidden_dim + m_hidden_dim) + 1, tagset_size)
        self.root2third = nn.Linear((p_hidden_dim + m_hidden_dim) + 1, tagset_size)
        
        self.p_hidden = self.init_hidden(p_hidden_dim)
        self.m_hidden = self.init_hidden(m_hidden_dim)

    def init_one_hot(self, vocab_size):
        #initialize each embedding
        #stack them together (or other ways to have  pretrained embeddings)
        #pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
        #we should turn data into torch.from_numpy(pretrained_weight)
        #embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
        torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
    
    def init_hidden(self, hidden_dim):
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, self.batch_size, hidden_dim),
                torch.zeros(1, self.batch_size, hidden_dim))

    def forward(self, prev, melody, mask):
        
        prev_embeds = self.p_embed(prev)
        x1 = torch.transpose(prev_embeds, 0, 1) #TODO: DEBUG THIS LINE (probably the last batch)
        
        melody_embeds =  self.m_embed(melody) #Error here
        x2 = torch.transpose(melody_embeds, 0, 1)
        p_lstm_out, self.p_hidden = self.p_lstm(x1, self.p_hidden)
        m_lstm_out, self.m_hidden = self.m_lstm(x2, self.m_hidden)
        
        p_fstate, m_fstate = p_lstm_out[-1], m_lstm_out[-1]
        

        concat = torch.cat((p_fstate, m_fstate), 1)
        #print(p_fstate.size(), m_fstate.size())
        tag_space = self.hidden2tag(concat) #REASON: you only need the final state
        tag_scores = F.log_softmax(tag_space, dim = 1)
        

        withroot = torch.cat((concat, tag_scores.max(1)[1].float().view(-1,1)), 1)
        second = self.root2second(withroot)
        third = self.root2third(withroot)
        second_scores = F.log_softmax(second, dim=1)
        third_scores = F.log_softmax(third, dim=1)
        
        #stacked = torch.stack([tag_scores, second_scores, third_scores], dim=0)

        return [tag_scores, second_scores, third_scores]

In [35]:
model = LSTMTagger(P_EMBEDDING_DIM, M_EMBEDDING_DIM, P_HIDDEN_DIM, M_HIDDEN_DIM, vocab_size, tagset_size, batch_size = batch_size)
loss_function = nn.NLLLoss(weight = newweights[0])
loss_function2 = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.2)

Train the model:



In [36]:
with torch.no_grad():
    for i in data_loader:
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        print("p, m, l", p, m, l)
        tag_scores = model(p, m, mask)
        print(tag_scores)
        break
    

p, m, l tensor([[  1,   4,   7,   6,   4,   7,   1,   4,   7,   8,   4,   7],
        [  1,   4,   7,  10,   3,   7,   3,   3,   7,   8,   3,   7],
        [ 10,   3,   7,   3,   4,   7,   8,   3,   7,   7,   4,   7],
        [ 11,   4,   7,   3,   3,   7,   4,   4,   7,   6,   4,   7]]) tensor([[  3,   3,   3,  12,  12,  12,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  8,   8,   8,   6,   6,   6,   6,   6,   6,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  1,   1, 

In [37]:
for epoch in range(ITER):  # again, normally you would NOT do 300 epochs, it is toy data
    for i in data_loader:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.p_hidden = model.init_hidden(P_HIDDEN_DIM)
        model.m_hidden = model.init_hidden(M_HIDDEN_DIM)

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        #print(m)
        tag_scores = model(p, m, mask)
        # Step 3. Run our forward pass.

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        #print(targets[:1])
        loss = 0
        
        loss += loss_function(tag_scores[0], torch.t(l)[0])  
        loss += 0.5 * loss_function2(tag_scores[1], torch.t(l)[1])
        loss += 0.25 * loss_function2(tag_scores[2], torch.t(l)[2])   
        
        loss.backward()
        optimizer.step()
        break
    if(epoch % 10==0): print(epoch, loss)

accuracy = 0.0
c = 0
#print("new")
for j in range(100):
    for i in data_loader:
        c+=1
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        tag_scores = model(p, m, mask)
        #print(tag_scores[0].size())
        pred = tag_scores[0].max(1)[1].numpy()
        truth = torch.t(l)[0].numpy()
        #print((3 == truth))
        #print(pred[30:50], truth[30:50])
        accuracy += np.average(pred == truth)
        if (c>=1): break
print(accuracy/c)


# See what the scores are after training
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(inputs)

#     # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#     # for word i. The predicted tag is the maximum scoring tag.
#     # Here, we can see the predicted sequence below is 0 1 2 0 1
#     # since 0 is index of the maximum value of row 1,
#     # 1 is the index of maximum value of row 2, etc.
#     # Which is DET NOUN VERB DET NOUN, the correct sequence!
#     _, point2 = tag_scores.max(1)
#     print(point2)

0 tensor(4.5248)
10 tensor(3.2804)
20 tensor(2.6759)
30 tensor(3.5713)
40 tensor(2.2532)
50 tensor(2.5057)
60 tensor(3.3110)
70 tensor(3.9084)
80 tensor(3.4953)
90 tensor(2.6497)
100 tensor(3.0828)
110 tensor(3.7762)
120 tensor(3.3194)
130 tensor(2.8022)
140 tensor(2.8333)
150 tensor(3.0554)
160 tensor(2.6764)


KeyboardInterrupt: 

In [ ]:
accuracy = 0.0
c = 0
#print("new")
for j in range(100):
    for i in data_loader:
        c+=1
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        tag_scores = model(p, m, mask)
        #print(tag_scores[0].size())
        pred = tag_scores[0].max(1)[1].numpy()
        truth = torch.t(l)[0].numpy()
        #print((3 == truth))
        #print(pred[30:50], truth[30:50])
        accuracy += np.average(pred == truth)
        if (c>=1): break
print(accuracy/c)